# Scraping and storing all pdfs related to "Eidgenössische Steuerverwaltung"

Work in progress... 

In [4]:
import urllib
import requests
import time
from bs4 import BeautifulSoup
from re import compile, sub, match, findall

In [5]:
url = "https://www.admin.ch/opc/de/federal-gazette/2015/index_32.html"

On this website you can find pdfs of the The Federal Gazette, which is a publication of the Swiss government. The pdfs related to "Eidgenössische Steuerverwaltung" and "Amtshilfe" contain names of foreign people, which are asked to get in touch immediately with the Swiss government. It is probably a matter of tax fraud.


In [6]:
pdflinks = []

url = "https://www.admin.ch/opc/de/federal-gazette/2015/index_32"
page = BeautifulSoup(requests.get(url,headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:5.0)","Referer":"http://google.com"}).text )

for year in range(2011,2016):
    for week in range(1,53):
        url = "https://www.admin.ch/opc/de/federal-gazette/"+str(year)+"/index_"+str(week)
        page = BeautifulSoup(requests.get(url,headers={}).text )

        table = page.find("table","table table-striped")
        if table:
            trs = table.find_all("tr")
            for row in trs:
                doclink = row.find("td", attrs = {"class":"nowrap"})
                if doclink:
                    title = doclink.find_next_sibling("td")
                    if title:
                        if "schluss" in title.getText().lower() and "amtshilfe" in title.getText().lower() or "estv" in title.getText().lower() and "amtshilfe" in title.getText().lower():
                            link = doclink.find("a")
                            filelink = "http://www.admin.ch"+urllib.unquote(link.get("href")).replace("&amp;","&")
                            filename = link.text
                            print filelink

                            #urllib.urlretrieve(filelink,"steuer/"+filename+".pdf") BLOCKED by the Swiss gov

                            pdflinks.append(filelink)
                            
                                #time.sleep(15)

http://www.admin.ch/opc/de/federal-gazette/2011/8765.pdf
http://www.admin.ch/opc/de/federal-gazette/2012/8005.pdf
http://www.admin.ch/opc/de/federal-gazette/2012/8032.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/1972.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/2537.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/2977.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/2979.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/3349.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/3449.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/4670.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/4672.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/4850.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/4857.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/5253.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/5255.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/7256.pdf
http://www.admin.ch/opc/de/federal-gazette/2013/7455.pdf
http://www.admin.ch/opc/de/fede

# New try with WGET

The Swiss gov blocked our scraper, ``urllib.urlretrieve`` didn't store real pdfs. Instead we got a message from the Swiss Gov: 

*"Dear Visitor
The federal authorities of Switzerland make use of a system scanning website requests for security violations. Your request has been flagged as such a violation and has therefore been blocked. It is possible that this constitutes a false positive. If you have received this message in error, please contact the service desk of the Federal Office for IT and Telecommunications, stating support-id D-3873271559171869965 as well as any additional information which may be relevant to the case. We will evaluate your case and report back to you within a week.
Federal Office of IT and Telecommunications"*

So, let's try WGET.

In [8]:
# ! brew install wget

In [ ]:
for pdflink in pdflinks:
    !wget -nc --random-wait -P steuer2 {pdflink}

--2015-08-27 12:40:15--  http://www.admin.ch/opc/de/federal-gazette/2011/8765.pdf
Resolving www.admin.ch... 162.23.128.199
Connecting to www.admin.ch|162.23.128.199|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.admin.ch/opc/de/federal-gazette/2011/8765.pdf [following]
--2015-08-27 12:40:15--  https://www.admin.ch/opc/de/federal-gazette/2011/8765.pdf
Connecting to www.admin.ch|162.23.128.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135530 (132K) [application/pdf]
Saving to: 'steuer2/8765.pdf'

8765.pdf            100%[=====================>] 132.35K   422KB/s   in 0.3s   

2015-08-27 12:40:16 (422 KB/s) - 'steuer2/8765.pdf' saved [135530/135530]

--2015-08-27 12:40:16--  http://www.admin.ch/opc/de/federal-gazette/2012/8005.pdf
Resolving www.admin.ch... 162.23.128.199
Connecting to www.admin.ch|162.23.128.199|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.admin.ch/opc/d

## Hooray! WGET stores all the pdfs we want in our local folder.

In [4]:
# Next step: parsing > pdf to text

In [7]:
! cd steuer

In [18]:
! for file in steuer/*.pdf; do pdftotext -enc UTF-8 "$file" "$file.txt"; done 

# Next step: parsing names with REGEX

In [ ]:
# dummy code!!
import re
def extract_name(string):
    print "Searching with regex #1"
    result = re.search(" (.*) ", string)
    if result:
        print "Found result, returning"
        return result.group(1)
    print "Didn't find anything, trying with regex #2"
    result = re.search("my name's (.*) and I am", string)
    if result:
        print "Found result, returning"
        return result.group(1)
    
my_content = "hello, my name"
my_content2 = "hello, my name"
print extract_name(my_content)
print "------"
print extract_name(my_content2)
print "------"

# NEXT STEP???

Cron job? Clustering? NLP? 